# 经济规划

## 目标和前提条件

在本示例中，您将了解如何使用数学优化来解决一个国家可能面临的宏观经济规划问题。我们将向您展示如何对涵盖一国经济不同部门之间相互关系的投入产出问题进行建模和求解。

此模型是H. Paul Williams编写的《Model Building in Mathematical Programming》第五版第263-264页和316-317页的示例9。

这是一个中级建模示例，我们假设您了解Python并熟悉Gurobi Python API。此外，您还应该具备一些构建数学优化模型的知识。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 问题描述

在这个问题中，我们假设有一个包含三类产业的经济体系：

* 煤炭
* 钢铁
* 运输

这些产业的部分产出需要作为其他产业的投入。例如，煤炭需要用于炼钢的高炉，钢铁需要用于开采煤炭的机械等。我们用货币单位衡量所有生产单位。也就是说，一个产业每生产一美元的产出，可能需要来自其自身产业以及其他产业的投入（以美元计）。下表显示了所需的投入以及劳动力需求（也以美元计）。

| 投入(t) / <br /> 产出(t+1) | 煤炭 | 钢铁 | 运输 |
| --- | --- | --- | --- | 
| 煤炭 | 0.1 | 0.5 | 0.4 | 
| 钢铁 | 0.1 | 0.1 | 0.2 |
| 运输 | 0.2 | 0.1 | 0.2 |
| 劳动力 | 0.6 | 0.3 | 0.2 | 
|**总计**  | **1** | **1** | **1**|

经济中存在时间滞后，因此t+1年的产出需要t年的投入。例如，生产价值一美元的煤炭需要0.1美元的煤炭（提供必要的能源）、0.1美元的钢铁（机械'磨损'所'损耗'的钢铁）和0.2美元的运输（用于从矿区运输煤炭）。此外，还需要0.6美元的人工。同样，表中的其他列给出了每生产一美元的钢铁和每生产一美元的运输（卡车、汽车、火车等）所需的投入（以美元计）。注意，每单位产出的价值恰好等于其投入价值的总和。

产业的产出也可用于在未来几年为自身或其他产业构建生产能力。下表给出了单位产能增加（以美元计的额外生产能力）所需的投入。t年的投入将在t+2年带来（永久性）产能增加。

| 投入(t) / <br /> 产出(t+2) | 煤炭 | 钢铁 | 运输 |
| --- | --- | --- | --- | 
| 煤炭 | 0.1 | 0.7 | 0.9 | 
| 钢铁 | 0.1 | 0.1 | 0.2 |
| 运输 | 0.2 | 0.1 | 0.2 |
| 劳动力 | 0.4 | 0.2 | 0.1 | 

商品可以从一年存储到下一年。目前（第1年初），存货和生产能力（每年）如下表所示（单位：百万美元）。

| 目前 | 存货 | 生产能力 |
| --- | --- | --- | 
| 煤炭 | 150 | 300 | 
| 钢铁 | 80 | 350 | 
| 运输 | 100 | 280 |  

我们的目标是在规划期内最大化总劳动力利用率（即就业），同时在每年满足以下外生消费需求：

* 煤炭6000万美元
* 钢铁6000万美元
* 运输3000万美元


## 模型构建
一种广泛使用的国民经济模型是投入产出模型，它表示一个国家经济不同部门之间的相互关系。这类模型通常被称为动态列昂惕夫模型，以其创始人的名字命名，他为美国经济构建了这样一个模型。Wagner (1957)也研究过类似的模型。

### 集合和索引

$i,j \in \text{Industries}=\{\text{coal}, \text{steel}, \text{transport}\}$

$ t \in \text{Horizon} = \{\text{year1}, \text{year2}, \text{year3}, \text{year4}, \text{year5}, \text{year6} \}$ 

$ t \in \text{fiveYears} = \{\text{year1}, \text{year2}, \text{year3}, \text{year4}, \text{year5} \}$  


$t \in H_{2,4} = \{year2, year3, year4 \} $

### 参数

$\text{demand}_{j} \in \mathbb{R}^+$: 产业 $j$ 的外生需求。

$\text{initial_stock}_{j} \in \mathbb{R}^+$: 第1年初产业 $j$ 的可用存货。

$\text{in_out_prod}_{i,j} \in \mathbb{R}^+$: 生产一单位产业 $j$ 的产品在下一年所需的产业 $i$ 的投入。

$\text{in_out_cap}_{i,j} \in \mathbb{R}^+$: 当年产业 $i$ 的投入导致两年后产业 $j$ 的生产能力永久性增加。

$\text{industry_cap}_{j} \in \mathbb{R}^+$: 第1年初产业 $j$ 的生产能力。

$\text{labor_prod}_{j} \in \mathbb{R}^+$: 产业 $j$ 生产商品所需的劳动力。

$\text{labor_extra_cap}_{j} \in \mathbb{R}^+$: 永久性增加产业 $j$ 产能所需的劳动力。

### 决策变量

$\text{production}_{j,t} \in \mathbb{R}^+$: t年可用的产业 $j$ 的商品生产量，单位为百万美元。
注：t年可用但在前一年生产的产业j的商品，单位为百万美元。

$\text{stock}_{j,t} \in \mathbb{R}^+$: t年末产业 $j$ 的存货水平，单位为百万美元。

$\text{extra_cap}_{j,t} \in \mathbb{R}^+$: t年初生效的产业 $j$ 的额外产能，单位为百万美元。


为了构建一个现实的模型，我们需要考虑第5年以后的情况。因此，我们做出以下假设：

* 外部需求在第5年及以后保持不变。
* 存货水平在第5年及以后保持不变。
* 第5年后不能增加产能。

因此，对于第6年及以后，我们可以假设一个静态经济模型。对于这个静态模型，每个产业 $i \in \text{Industries}$ 的商品生产量可以按如下方式计算。

\begin{equation}
x_{i} = \text{demand}_{i} + \sum_{j \in \text{Industries} } \text{in_out_prod}_{i,j} * x_{j}
\end{equation}

其中 $x_{i}$ 是产业 $i$ 的商品生产量。

求解这个方程组可以得到第5年以后各年商品生产量的下界。然后，第6年及以后产业 $j \in \text{Industries}$ 的商品生产量可以由以下约束定义。

$$
\text{production}_{j,t} \geq x_{j} \quad \forall j \in \text{Industries}, \; t=6. 
$$

$$
\text{extra_cap}_{j,t} = 0 \quad \forall j \in \text{Industries}, \; t=6.
$$


### 约束条件

**第1年平衡方程**：产业 $i$ 的初始存货应等于总需求（内部和外部）、需要建设的额外产能、以及该产业第1年末的存货水平之和。

\begin{equation}
\text{initial_stock}_{i} = \sum_{j \in \text{Industries} } \text{in_out_prod}_{i,j}*\text{production}_{j,2}
+ \text{demand}_{i} + \sum_{j \in \text{Industries} } \text{in_out_cap}_{i,j}*\text{extra_cap}_{j,3} +
 \text{stock}_{i,1}
\end{equation}

**第2年、第3年和第4年的平衡方程**：t年产业 $i$ 的商品生产量加上t-1年末的存货应等于总需求（内部和外部）、需要建设的额外产能、以及该产业t年末的存货水平之和。


\begin{equation}
\text{production}_{j,t} + \text{stock}_{i,t-1} =
\sum_{j \in \text{Industries} } \text{in_out_prod}_{i,j}*\text{production}_{j,t+1} + \text{demand}_{i} +
\sum_{j \in \text{Industries} } \text{in_out_cap}_{i,j}*\text{extra_cap}_{j,t+2} 
+ \text{stock}_{i,t} \quad \forall t \in H_{2,4}
\end{equation}


**第5年平衡方程**：产业 $i$ 的商品生产量加上第4年末的存货应等于总需求（内部和外部）加上该产业第5年末的存货水平。

\begin{equation}
\text{production}_{j,5} + \text{stock}_{i,4} =
\sum_{j \in \text{Industries} } \text{in_out_prod}_{i,j}*\text{production}_{j,6}  +
\text{demand}_{i} + \text{stock}_{i,5}
\end{equation}


**规划期末约束**：

$$
\text{production}_{j,t} \geq x_{j} \quad \forall j \in \text{Industries}, \; t=6 
$$

$$
\text{extra_cap}_{j,t} = 0 \quad \forall j \in \text{Industries}, \; t=6
$$

其中 $x_{j}$ 是静态模型的解。


**生产能力约束**：这些约束确保规划期内各产业的商品生产量不超过该年的总生产能力。

\begin{equation}
\text{production}_{j,t} \leq \text{base_cap}_{j} + \sum_{\tau \leq t} \text{extra_cap}_{j,\tau} \quad \forall t \in \text{Horizon}
\end{equation}

### 目标函数

**劳动力利用**：最大化就业。

\begin{equation}
\text{最大化} \quad Z =
\sum_{t \in \text{fiveYears} } \sum_{j \in \text{Industries} } \text{labor_prod}_{j}*\text{production}_{j,t} +
\sum_{t \in \text{fiveYears} } \sum_{j \in \text{Industries} } \text{labor_extra_cap}_{j}*\text{extra_cap}_{j,t}
\end{equation}


---
## Python实现

我们导入Gurobi Python模块和其他Python库。

In [ ]:
# %pip install gurobipy

In [1]:
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

## 输入数据
我们定义模型的所有输入数据。

In [ ]:
# 每个产业生产商品的投入产出矩阵

arcs, inout_prod = gp.multidict({
    ('coal', 'coal'): 0.1,
    ('coal', 'steel'): 0.5,
    ('coal', 'transport'): 0.4,
    ('steel', 'coal'): 0.1,
    ('steel', 'steel'): 0.1,
    ('steel', 'transport'): 0.2,
    ('transport', 'coal'): 0.2,
    ('transport', 'steel'): 0.1,
    ('transport', 'transport'): 0.2
})

# 每个产业生产商品所需的劳动力
labor_prod = dict({'coal': 0.6,
                   'steel': 0.3,
                   'transport': 0.2})

# 为每个产业创建额外产能的投入产出矩阵

arcs, inout_cap = gp.multidict({
    ('coal', 'coal'): 0.1,
    ('coal', 'steel'): 0.7,
    ('coal', 'transport'): 0.9,
    ('steel', 'coal'): 0.1,
    ('steel', 'steel'): 0.1,
    ('steel', 'transport'): 0.2,
    ('transport', 'coal'): 0.2,
    ('transport', 'steel'): 0.1,
    ('transport', 'transport'): 0.2
})

# 为每个产业增加产能所需的劳动力
labor_extra_cap = dict({'coal': 0.4,
                  'steel': 0.2,
                  'transport': 0.1})

# 每个产业的初始存货、初始产能和商品需求

industries, stock0, capacity0, demand = gp.multidict({
    'coal': [250,300,60],
    'steel': [180,350,60],
    'transport': [200,280,30]
})

# 时间范围
horizon = [1,2,3,4,5,6]
fiveYears = [1,2,3,4,5]
years2_4 = [2,3,4]

# 计算参数
i2h = set(product(industries, horizon))
i2f = set(product(industries, fiveYears))

## 预处理

我们假设第6年及以后采用静态经济模型。这个静态模型由一系列方程定义。该模型的决策变量是静态经济模型中各产业的商品生产量。我们对这个模型进行构建和求解。

In [ ]:
static = gp.Model('StaticModel')

static_prod = static.addVars(industries, name="static_prod")

# 静态模型平衡方程

static_balance = static.addConstrs(
    (static_prod[i] - gp.quicksum(inout_prod[i,j]*static_prod[j] for j in industries) 
     == demand[i] for i in industries  ), name='static_balance' )

# 我们定义一个常数目标函数来求解静态模型的平衡方程组

static.setObjective(0)

# 验证模型构建

static.write("StaticModel.lp")

# 运行优化引擎

static.optimize()

# 打印静态模型的解

print("\n\n_________________________________________________________________________________")
print(f"经济静态模型中各产业的商品生产量为:")
print("_________________________________________________________________________________")
for i in industries:
    if (static_prod[i].x > 1e-6):
        dollars_static_prod = '${:,.2f}'.format(static_prod[i].x)
        print(f"生产{i}商品 {dollars_static_prod} 百万美元")

Using license file c:\gurobi\gurobi.lic
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x5b989667
Coefficient statistics:
  Matrix range     [1e-01, 9e-01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+01]
Presolve removed 3 rows and 3 columns
Presolve time: 0.05s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds
Optimal objective  0.000000000e+00


_________________________________________________________________________________
The production of goods by industry for the static model of the economy is:
_________________________________________________________________________________
Generate $166.40 million dollars of coal

## 模型部署

我们创建一个模型和相关变量。对于规划期内的每一年和每个产业，这些变量包括商品生产量、商品库存水平和产能增加量。

In [ ]:
model = gp.Model('EconomicPlanning')

# 决策变量
production = model.addVars(i2h, ub=capacity0, name="production")

# 第1年没有生产
model.setAttr('ub', production.select('*',1), 0)    

stock = model.addVars(i2f, name="stock")
extra_cap = model.addVars(i2h, name="extra_cap")


# 第1年和第2年不能增加产能
for j,t in i2h:
    if t < 3:
        extra_cap[j,t].ub = 0  

第一个约束是规划期第1年的平衡方程。产业 $i$ 的初始存货应等于总需求（内部和外部）、需要建设的额外产能、以及该产业第1年末的存货水平之和。

In [ ]:
# 第1年平衡方程

balance1 = model.addConstrs( ( stock0[i] == gp.quicksum(inout_prod[i,j]*production[j,2] for j in industries)  
                              + gp.quicksum(inout_cap[i,j]*extra_cap[j,3] for j in industries ) 
                              + demand[i] + stock[i,1] for i in industries ), name='balance1' )

以下约束是第2年、第3年和第4年的平衡方程。t年产业 i 的商品生产量加上t-1年末的存货应等于总需求（内部和外部）、需要建设的额外产能、以及该产业t年末的存货水平之和。

In [ ]:
# 第2年、第3年和第4年的平衡方程

balance_t = model.addConstrs(( production[i, year] + stock[i,year-1]  == 
                              gp.quicksum(inout_prod[i,j]*production[j, year + 1] for j in industries) 
                              + gp.quicksum(inout_cap[i,j]*extra_cap[j, year + 2] for j in industries ) 
                              + demand[i] + stock[i, year] for i in industries for year in years2_4 ), name='balance_t' )

第5年的约束确保产业 i 的商品生产量加上第4年末的存货应等于总需求（内部和外部）加上该产业第5年末的存货水平。

In [ ]:
# 第5年平衡方程

balance5 = model.addConstrs( (production[i, 5] + stock[i,4]  == 
                              gp.quicksum(inout_prod[i,j]*production[j,6] for j in industries) 
                              + demand[i] + stock[i,5] for i in industries ), name='balance5' )

我们需要对第6年及以后的静态经济模型实施约束。
* 每个产业的商品生产量应大于或等于静态经济模型中的产业生产量。

* 第6年及以后不再增加产能。

In [ ]:
# 第6年及以后的稳态生产

steadyProduction = model.addConstrs((production[j,6] - static_prod[j].x >= 0 for j in industries ), name='steadyProduction')

# 第6年及以后不再增加产能
for j,t in i2h:
    if t == 6:
        extra_cap[j,t].ub = 0   

生产能力约束确保规划期内各产业的商品生产量不超过该年的总生产能力。

In [ ]:
# 生产能力约束

capacityConstr = model.addConstrs(
    (production[industry, year] - gp.quicksum(extra_cap[industry,t] for t in fiveYears if t <= year) 
     <= capacity0[industry] for industry,year in i2f ), name='capacityConstr' )

目标是最大化劳动力就业。

In [ ]:
# 最大化就业
model.setObjective(
    (gp.quicksum(labor_prod[j]*production[j,t] for j in industries for t in fiveYears) 
     + gp.quicksum(labor_extra_cap[j]*extra_cap[j,t] for j in industries for t in fiveYears) ), GRB.MAXIMIZE)

In [ ]:
# 验证模型构建

model.write('DynamicModel.lp')

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 33 rows, 51 columns and 183 nonzeros
Model fingerprint: 0x194e5768
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e-01, 6e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 4e+02]
Presolve removed 9 rows and 18 columns
Presolve time: 0.01s
Presolved: 24 rows, 33 columns, 126 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5258351e+04   7.671320e+03   0.000000e+00      0s
      25    1.9022169e+03   0.000000e+00   0.000000e+00      0s

Solved in 25 iterations and 0.01 seconds
Optimal objective  1.902216910e+03


---
## 分析

最大化就业的目标导致总劳动力利用率达到19.0222亿美元。在这个目标下，由于煤炭产业的劳动力需求较高，该产业得到了提升。

In [ ]:
# 输出报告

print("_______________________________________________________________________________________________")
print(f"动态列昂惕夫经济模型中各产业按年份的商品生产量为:")
print("_______________________________________________________________________________________________")

goods = {}
 
for i in industries:
    my_list = []
    for t in fiveYears:
        my_list.append('${:,.2f}'.format( production[i,t ].x ) )
    goods[i] = my_list

goods_production = pd.DataFrame(goods, index=["第1年", "第2年", "第3年", "第4年", "第5年"])
goods_production

_______________________________________________________________________________________________
The production of goods by industry and year for the dynamic Leontief model of the economy is:
_______________________________________________________________________________________________


,coal,steel,transport
Year1,$0.00,$0.00,$0.00
Year2,$300.00,$136.78,$211.30
Year3,$370.87,$231.91,$217.39
Year4,$368.03,$209.57,$275.67
Year5,$961.06,$350.00,$92.31


In [ ]:
print("_______________________________________________________________________________________________")
print(f"动态列昂惕夫经济模型中各产业按年份的生产能力为:")
print("_______________________________________________________________________________________________")

# 计算累积产能
totalCap = {}
 
for i in industries:
    amount = capacity0[i]
    my_list = []
    for t in fiveYears:
        amount += extra_cap[i,t].x
        my_list.append(round(amount))
    totalCap[i] = my_list
    
extra_capacity = pd.DataFrame(totalCap, index=["第1年", "第2年", "第3年", "第4年", "第5年"])
extra_capacity

_______________________________________________________________________________________________
The productive capacity by industry and year for the dynamic Leontief model of the economy is:
_______________________________________________________________________________________________


,coal,steel,transport
Year1,300,350,280
Year2,300,350,280
Year3,371,350,280
Year4,371,350,280
Year5,961,350,280


In [ ]:
print("____________________________________________________________________________________")
print(f"动态列昂惕夫经济模型中各产业年末的存货水平为:")
print("____________________________________________________________________________________")

inv = {}
 
for i in industries:
    my_list = []
    for t in fiveYears:
        my_list.append('${:,.2f}'.format( stock[i,t ].x ) )
    inv[i] = my_list

stock_level = pd.DataFrame(inv, index=["第1年", "第2年", "第3年", "第4年", "第5年"])
stock_level

____________________________________________________________________________________
Stock level by industry at the end of year for the dynamic Leontif model of the economy is:
____________________________________________________________________________________


,coal,steel,transport
Year1,$0.00,$26.97,$39.89
Year2,$0.00,$0.00,$80.34
Year3,$0.00,$0.00,$0.00
Year4,$0.00,$0.00,$0.00
Year5,$794.66,$244.33,$0.00


## 参考文献

H. Paul Williams, Model Building in Mathematical Programming, 第五版。

Wagner, H.M. (1957) A linear programming solution to dynamic Leontief type models. Management Science, 3, 234–254。

Copyright © 2020 Gurobi Optimization, LLC